<a href="https://colab.research.google.com/github/Habib-20/kel1-project-apm/blob/main/RuleBasedClassifier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv(r"persona.csv")
df.head(5)

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [3]:
agg_df = df.groupby(['COUNTRY', 'SOURCE', 'SEX', 'AGE'])[["PRICE"]].aggregate("mean")
agg_df= agg_df.sort_values('PRICE', ascending=False)
agg_df = agg_df.reset_index()
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


In [4]:
my_labels = ['0_18', '19_23', '24_30', '31_40', '41_70']
agg_df["AGE_CUT"] = pd.cut(x=agg_df["AGE"], bins=[0, 18, 23, 30, 40, 70], labels=my_labels)
agg_df.tail(10) 

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CUT
338,bra,android,male,23,21.5,19_23
339,tur,android,male,21,19.0,19_23
340,tur,ios,male,47,19.0,41_70
341,bra,ios,female,34,19.0,31_40
342,bra,ios,male,47,19.0,41_70
343,usa,ios,female,38,19.0,31_40
344,usa,ios,female,30,19.0,24_30
345,can,android,female,27,19.0,24_30
346,fra,android,male,18,19.0,0_18
347,deu,android,male,26,9.0,24_30


In [6]:
agg_df["customers_level_based"] = [f"{i[0]}_{i[1]}_{i[2]}_{i[-1]}" for i in agg_df.values]


In [7]:
agg_df = agg_df.loc[:, ["customers_level_based", "PRICE"]].groupby("customers_level_based").agg({"PRICE": "mean"}).sort_values(by="PRICE", ascending=False).reset_index()
agg_df["customers_level_based"].head() 

0    fra_android_female_fra_android_female_24_30
1                tur_ios_male_tur_ios_male_24_30
2                tur_ios_male_tur_ios_male_31_40
3    tur_android_female_tur_android_female_31_40
4        can_android_male_can_android_male_19_23
Name: customers_level_based, dtype: object

In [9]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df

,customers_level_based,PRICE,SEGMENT
0,fra_android_female_fra_android_female_24_30,45.428571,A
1,tur_ios_male_tur_ios_male_24_30,45.000000,A
2,tur_ios_male_tur_ios_male_31_40,42.333333,A
3,tur_android_female_tur_android_female_31_40,41.833333,A
4,can_android_male_can_android_male_19_23,40.111111,A
...,...,...,...
104,tur_ios_male_tur_ios_male_41_70,25.052632,D
105,tur_ios_male_tur_ios_male_19_23,24.833333,D
106,fra_android_male_fra_android_male_0_18,24.625000,D
107,can_android_female_can_android_female_24_30,19.000000,D


In [11]:
agg_df.groupby(["SEGMENT"]).agg({"PRICE": ["mean", "max", "sum"]})

agg_df[agg_df["SEGMENT"] == "C"].describe()

,PRICE
count,27.000000
mean,33.509674
std,0.492587
min,32.500000
25%,33.000000
50%,33.627634
75%,34.000000
max,34.077340
